In [1]:
cd ~/Desktop/DeepLearningPilot/

/Users/jz-rolling/Desktop/DeepLearningPilot


In [2]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from skimage import filters,morphology,measure
from scipy.stats import pearsonr
import seaborn as sns
from ncolor import label
from timeit import default_timer
import pickle as pk, glob, os
import momia2 as mo
from ncolor import label
import tifffile
from matplotlib import cm
import cv2

In [66]:
sample_dict = {}
for i in [7,8,9]:
    for j,k in enumerate(sorted(glob.glob('/Volumes/JZSSD_temp/20190709_NQTF/pred_results/{}_*tracker.pk'.format(i)))):
        header = k.split('_tracker.pk')[0]
        phase = np.load(header+'_clips.npy')
        masks = np.load(header+'_omni_masks.npy')
        fitc = np.load(header+'_fitc.npy')
        merged_rp = pk.load(open(k,'rb'))
        sample_dict['{}_{}'.format(i,j)] = [phase,fitc,merged_rp,k,masks]

In [67]:
files = sorted(glob.glob('/Volumes/JZSSD_temp/20190709_NQTF/3_kymographs/INH_selected/*.png'))
for f in files:
    roi = '_'.join(f.split('/')[-1].split('_')[:2])
    cid = f.split('/')[-1].split('_')[-1].split('.png')[0]
    subf = '/Volumes/JZSSD_temp/20190709_NQTF/3_kymographs/INH_selected/{}_{}/'.format(roi,cid)
    if not os.path.isdir(subf):
        os.mkdir(subf)
    phase=sample_dict[roi][0]
    fitc= sample_dict[roi][1]
    subset = sample_dict[roi][2][sample_dict[roi][2]['cell_lineage']==cid].copy()
    contours = subset['$refined_contours'].values
    x1 = subset['$opt-x1'].values
    y1 = subset['$opt-y1'].values
    _x1 = subset['$opt-x1'].min()
    _x2 = subset['$opt-x2'].max()
    _y1 = subset['$opt-y1'].min()
    _y2 = subset['$opt-y2'].max()
    times = subset['$time'].values
    start_t = times[0]
    end_t = times[-1]
    h,w = _x2-_x1, _y2-_y1
    
    for t in range(start_t,end_t):
        if h>w:
            fig,axes=plt.subplots(figsize=(2*w/40,h/40),ncols=2,gridspec_kw={'wspace':0})
        else:
            fig,axes=plt.subplots(figsize=(w/40,2*h/40),nrows=2,gridspec_kw={'hspace':0})
            
        axes[0].imshow(phase[t,_x1:_x2,_y1:_y2],cmap='gist_gray',vmin=0,vmax=1)
        axes[1].imshow(fitc[t,_x1:_x2,_y1:_y2],cmap='gist_gray',vmin=950,vmax=2000)
        axes[0].axis('off')
        axes[1].axis('off')
        cx = contours[t-start_t][0][:,0]+x1[t-start_t]-_x1
        cy = contours[t-start_t][0][:,1]+y1[t-start_t]-_y1
        axes[0].plot(cy,cx,color='orange',lw=0.5)
        axes[1].plot(cy,cx,color='orange',lw=0.5)
        axes[0].plot([5,5+1/0.065],[5,5],color='w',lw=3)
        axes[0].text(0,-1,'{} hours'.format(t*0.25),color='black',va='bottom',ha='left',fontsize=12)
        plt.savefig(subf+'{}.png'.format(t),bbox_inches='tight')
        plt.close()

In [68]:
# render movies
for root in sorted(glob.glob('/Volumes/JZSSD_temp/20190709_NQTF/3_kymographs/*_selected/')):
    for f in sorted(glob.glob(root+'*/')):
        #f = '/Volumes/JZSSD_temp/20190709_NQTF/trace_plot/RIF/11_6/'
        header = f.split('/')[-2]
        frames = []

        for p in sorted(glob.glob(f+'*.png')):
            #p = f+'{}.png'.format(i)
            img = cv2.imread(p)
            height, width, _c = img.shape
            frames.append(img)
        fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
        out2 = cv2.VideoWriter(root+'{}.avi'.format(header),
                                fourcc, fps=6, frameSize=np.flip(img.shape[:2])) 
        for i in range(len(frames)):
            out2.write(frames[i])
        out2.release()